In [1]:
import os
import gzip
import json
import numpy as np
import pandas as pd
import tqdm
pd.set_option('display.max_rows', 100)

## Heroes information and stats

In [2]:
with open("../data/heroes.json", "r") as fp: 
    heroes = json.load(fp)
    
df_heroes = pd.DataFrame(heroes)
df_heroes

,id,name,localized_name,primary_attr,attack_type,roles,legs
0,1,npc_dota_hero_antimage,Anti-Mage,agi,Melee,"[Carry, Escape, Nuker]",2
1,2,npc_dota_hero_axe,Axe,str,Melee,"[Initiator, Durable, Disabler, Jungler, Carry]",2
2,3,npc_dota_hero_bane,Bane,int,Ranged,"[Support, Disabler, Nuker, Durable]",4
3,4,npc_dota_hero_bloodseeker,Bloodseeker,agi,Melee,"[Carry, Disabler, Jungler, Nuker, Initiator]",2
4,5,npc_dota_hero_crystal_maiden,Crystal Maiden,int,Ranged,"[Support, Disabler, Nuker, Jungler]",2
...,...,...,...,...,...,...,...
116,123,npc_dota_hero_hoodwink,Hoodwink,agi,Ranged,"[Support, Nuker, Escape, Disabler]",4
117,126,npc_dota_hero_void_spirit,Void Spirit,int,Melee,"[Carry, Escape, Nuker, Disabler]",2
118,128,npc_dota_hero_snapfire,Snapfire,str,Ranged,"[Support, Nuker, Disabler, Escape]",2
119,129,npc_dota_hero_mars,Mars,str,Melee,"[Carry, Initiator, Disabler, Durable]",2


In [4]:
df_heroes_dict = df_heroes[["id", "localized_name"]]
df_heroes_dict

,id,localized_name
0,1,Anti-Mage
1,2,Axe
2,3,Bane
3,4,Bloodseeker
4,5,Crystal Maiden
...,...,...
116,123,Hoodwink
117,126,Void Spirit
118,128,Snapfire
119,129,Mars


### Notes:

* We will look from the alliance side
* There are 121 heroes as of 7.30 patch
* Newest hero is Dawnbreaker
* 63 ranged, 58 melee
* 43 int, 40 str, 38 str

## Preprocessing loop

In [7]:
def get_true_index(i):
    idx = i
    if type(idx) == str:
        idx = int(i)
    
    try:
        true_index = df_heroes[df_heroes["id"] == idx].index[0]
    except IndexError as e:
        print(e)
        print("heroid:",i)
        raise IndexError
    
    return df_heroes[df_heroes["id"] == idx].index[0]

def get_hero_name_by_id(i):
    idx = i
    if type(idx) == str:
        idx = int(i)
    
    return df_heroes[df_heroes["id"].index == idx]["localized_name"].values[0]

def get_comp_by_training_sample(sample):
    radiant_win = sample[-1]
    radiant = []
    dire = []
    for i, h in enumerate(sample[:-1]):
        if h == 1:
            radiant.append(get_hero_name_by_id(i))
        elif h == -1:
            dire.append(get_hero_name_by_id(i))
    print(radiant, dire, radiant_win)

In [8]:
results = []
seen_matches = set()
dup_counter = 0
# [[1, 0, -1, ... , n_heroes_121, winner]]
count = 0
with gzip.open("../data/matches_5148330922-5148330922_905547samples.gz", "r") as fp:
    for line in tqdm.tqdm(fp):
#         if count > 2:
#             break
        training_sample = np.zeros((122,))
        match = json.loads(line)
        
        if not "match_id" in match:
            continue
        
        if len(match["players"]) != 10:
            continue
        
        if match["match_id"] in seen_matches:
            dup_counter += 1
            continue
        
        for p in match["players"]:
            hero_i = get_true_index(p["hero_id"])
            is_radiant = True if p["player_slot"] <= 7 else False # DETTE ER RIKTIG!!! 

            if is_radiant: # radiant slot
                training_sample[hero_i] = 1
            else: 
                training_sample[hero_i] = -1
            
        training_sample[-1] = match["radiant_win"]
        
#         get_comp_by_training_sample(training_sample)
        
        results.append(training_sample)
#         count += 1
        seen_matches.add(match["match_id"])
        
print("# duplicates", dup_counter)
print("# matches", len(seen_matches))
with open("../data/matches_5148330922-5148330922.npy", "wb") as fp:
    np.save(fp, np.array(results))

167230it [17:52, 155.87it/s]

index 0 is out of bounds for axis 0 with size 0
heroid: 0


IndexError: 